In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import subprocess
from subprocess import check_output
try:
    print(check_output(["ls", "../v2"]).decode("utf8"))
except subprocess.CalledProcessError as e:
    raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
# Any results you write to the current directory are saved as output.

frame
video



In [2]:
import tensorflow as tf

In [3]:
# now, let's read the frame-level data
# due to execution time, we're only going to read the first video
video_files = os.listdir("../v2/video/")
frame_files = os.listdir("../v2/frame/")

# Global variables
num_labels = 4716

In [4]:
def createTargetVec(labels):
    out = np.zeros((1, num_labels))
    for label in labels:
        out[0,label] = 1
    return out

In [5]:
def get_dataset(frames, samples=5, k=0):
    rgb_input = np.empty((samples, 100, 1024))
    audio_input = np.empty((samples, 100, 128))
    label_output = np.empty((samples, num_labels))
    sess = tf.InteractiveSession()
    for example in tf.python_io.tf_record_iterator(frames):        
        tf_seq_example = tf.train.SequenceExample.FromString(example)
        labels = tf_seq_example.context.feature['labels'].int64_list.value
        rgb_frame = np.zeros((100, 1024))
        audio_frame = np.zeros((100, 128))
        for i in range(100):
            rgb_frame[i] = tf.cast(tf.decode_raw(
                   tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0],tf.uint8)
                          ,tf.float32).eval()
            print("rgb_frame: ", rgb_frame[i])
            print("rgb_frame shape: ", rgb_frame[i].shape)
            audio_frame[i] = tf.cast(tf.decode_raw(
                    tf_seq_example.feature_lists.feature_list['audio'].feature[i].bytes_list.value[0],tf.uint8)
                           ,tf.float32).eval()
            print("rgb_frame: ", audio_frame[i])
            print("rgb_frame shape: ", audio_frame[i].shape)
            print("  ")
            print("  ")
            print("  ")
            print("  ")
#         print("rgb_frame: ", rgb_frame)
        rgb_input[k] = rgb_frame
#         print("rgb_input: ", rgb_input)
        audio_input[k] = audio_frame
        label_output[k] = createTargetVec(labels[:])
        k += 1
        progress = (k / samples) * 100
        if int(progress) % 10 == 0:
            print("Progress", progress, "%")
        if k >= samples:
            break
    sess.close()
    return audio_input, rgb_input, label_output

In [6]:
# video_lvl_record = "../v2/video/train-1.tfrecord"
frame_lvl_record = "../v2/frame/train4f.tfrecord"

audio_input, rgb_input, label_output = get_dataset(frame_lvl_record)

X_train_rgb, X_test_rgb, _, _ = train_test_split(rgb_input, label_output, test_size=0.2, random_state=42)
X_train_audio, X_test_audio, y_train, y_test = train_test_split(audio_input, label_output, test_size=0.2, random_state=42)

rgb_frame:  [100.  54.  84. ...  46.  10. 132.]
rgb_frame shape:  (1024,)
rgb_frame:  [148. 104. 159. 214. 151.  74. 126.  36. 186. 181.  93.  94.  83. 134.
 174.  87.  93. 104. 103. 182. 160. 123.  36.  91. 138. 192. 117. 193.
  70. 255. 199. 173.  12. 242.  91. 141.  99.  63.  54.  79.  79. 201.
 158. 177.  80. 121. 143.  58. 195. 139. 130. 194. 116. 130. 155.  36.
 141. 221.  85.  44. 194.  73. 152. 156.  37. 108.  21. 125. 133. 180.
  15. 191. 178. 174. 158. 226. 174.  44. 103.   0. 207.  92.  79. 193.
   0.  54. 119. 100.  69.  82. 175.  65. 104. 112. 255. 157. 199. 109.
 109. 193.  27. 250.  99. 122.  95.  52.   2. 139. 200.  97. 224. 132.
  96. 145.  22. 171.  37.  19. 212.  11. 170. 104. 158. 240. 122. 247.
  62. 149.]
rgb_frame shape:  (128,)
  
  
  
  
rgb_frame:  [105.   0.  91. ... 153.  27. 103.]
rgb_frame shape:  (1024,)
rgb_frame:  [120.  83. 186. 169. 169.  53. 120.  76. 200. 182. 102.  66. 105. 124.
 155.  86. 128. 176. 196. 151. 136. 173.  91. 142.  97. 175. 110. 205

In [7]:
X_train_rgb

array([[[  0.,  72., 173., ..., 142., 119., 137.],
        [  0.,  72., 173., ..., 142., 119., 137.],
        [  0.,  72., 173., ..., 142., 119., 137.],
        ...,
        [229., 153., 194., ..., 103., 120., 188.],
        [165., 165., 228., ..., 140., 101., 214.],
        [226., 151., 219., ..., 124.,  91.,  88.]],

       [[  0.,  85., 148., ..., 124., 135., 133.],
        [  0.,  62., 104., ..., 116., 111., 128.],
        [  0.,  75., 103., ...,  74.,  26., 176.],
        ...,
        [144., 172.,  86., ..., 244.,  72.,  95.],
        [210., 153., 100., ..., 255., 200., 189.],
        [200., 160.,  99., ..., 194., 220., 195.]],

       [[100.,  54.,  84., ...,  46.,  10., 132.],
        [105.,   0.,  91., ..., 153.,  27., 103.],
        [ 93.,  24.,  95., ...,  84., 113., 130.],
        ...,
        [ 90.,  35., 122., ..., 190., 155., 147.],
        [ 86.,  26., 112., ..., 129., 100., 126.],
        [ 81.,  12., 120., ..., 222., 114., 140.]],

       [[161.,  57.,  93., ...,  44.,

In [8]:
X_train_rgb.shape

(4, 100, 1024)

In [9]:
X_train_audio

array([[[173.,  27., 126., ..., 133., 130., 187.],
        [173.,  27., 126., ..., 133., 130., 187.],
        [182.,  67.,  94., ..., 133.,  82., 137.],
        ...,
        [216.,  99., 127., ...,  83., 184., 154.],
        [203.,  91., 135., ..., 101.,  76.,  75.],
        [197., 139., 162., ..., 189., 185., 122.]],

       [[173.,  27., 126., ..., 133., 130., 187.],
        [165.,  94., 111., ...,  57., 155., 255.],
        [164.,  96.,  89., ..., 199., 176., 154.],
        ...,
        [182., 109.,  92., ..., 175., 255.,   0.],
        [188., 108., 106., ..., 118., 192.,   0.],
        [188., 127., 100., ..., 160.,  24.,   0.]],

       [[148., 104., 159., ..., 247.,  62., 149.],
        [120.,  83., 186., ..., 182.,  61., 180.],
        [118.,  75., 117., ..., 195.,  85.,  59.],
        ...,
        [ 43., 210.,   5., ...,  55., 252., 236.],
        [118.,  75., 133., ..., 159.,   0., 240.],
        [ 33., 218.,   8., ...,  15., 255., 221.]],

       [[102.,  87., 123., ...,  57.,

In [10]:
X_train_audio.shape

(4, 100, 128)

In [11]:
# rgb: 1024
# audio: 128
x_train_np = np.concatenate((X_train_rgb, X_train_audio), axis=2)

In [12]:
x_train_np.shape

(4, 100, 1152)

In [13]:
x_test_np = np.concatenate((X_test_rgb, X_test_audio), axis=2)

In [14]:
x_test_np.shape

(1, 100, 1152)

In [15]:
x_train_rgb_ = np.split(x_train_np, [128], axis=2)

In [16]:
x_train_rgb_[0].shape

(4, 100, 128)

# 

In [17]:
print(len(X_train_rgb), 'train rgb')
print(len(X_test_rgb), 'test rgb')
print("")
print(len(X_train_audio), 'train sequences')
print(len(X_test_audio), 'test sequences')
print("")
print('X_train_rgb shape:', X_train_rgb.shape)
print('X_test_rgb shape:', X_test_rgb.shape)
print("")
print('X_train_audio shape:', X_train_audio.shape)
print('X_test_audio shape:', X_test_audio.shape)
print("")
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

4 train rgb
1 test rgb

4 train sequences
1 test sequences

X_train_rgb shape: (4, 100, 1024)
X_test_rgb shape: (1, 100, 1024)

X_train_audio shape: (4, 100, 128)
X_test_audio shape: (1, 100, 128)

y_train shape: (4, 4716)
y_test shape: (1, 4716)


In [18]:
X_train = [X_train_rgb, X_train_audio]
X_test = [X_test_rgb, X_test_audio]

In [19]:
# 2
# 4
# 100
# 1024
len(X_train[1])

4

In [20]:
X_train_rgb

array([[[  0.,  72., 173., ..., 142., 119., 137.],
        [  0.,  72., 173., ..., 142., 119., 137.],
        [  0.,  72., 173., ..., 142., 119., 137.],
        ...,
        [229., 153., 194., ..., 103., 120., 188.],
        [165., 165., 228., ..., 140., 101., 214.],
        [226., 151., 219., ..., 124.,  91.,  88.]],

       [[  0.,  85., 148., ..., 124., 135., 133.],
        [  0.,  62., 104., ..., 116., 111., 128.],
        [  0.,  75., 103., ...,  74.,  26., 176.],
        ...,
        [144., 172.,  86., ..., 244.,  72.,  95.],
        [210., 153., 100., ..., 255., 200., 189.],
        [200., 160.,  99., ..., 194., 220., 195.]],

       [[100.,  54.,  84., ...,  46.,  10., 132.],
        [105.,   0.,  91., ..., 153.,  27., 103.],
        [ 93.,  24.,  95., ...,  84., 113., 130.],
        ...,
        [ 90.,  35., 122., ..., 190., 155., 147.],
        [ 86.,  26., 112., ..., 129., 100., 126.],
        [ 81.,  12., 120., ..., 222., 114., 140.]],

       [[161.,  57.,  93., ...,  44.,

## Lets take a look at the rgb data
****

In [21]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [22]:
rgb_input.shape

(5, 100, 1024)

In [23]:
rgb_input[0]

array([[100.,  54.,  84., ...,  46.,  10., 132.],
       [105.,   0.,  91., ..., 153.,  27., 103.],
       [ 93.,  24.,  95., ...,  84., 113., 130.],
       ...,
       [ 90.,  35., 122., ..., 190., 155., 147.],
       [ 86.,  26., 112., ..., 129., 100., 126.],
       [ 81.,  12., 120., ..., 222., 114., 140.]])

In [24]:
rgb_input[0].shape

(100, 1024)

In [25]:
print('The first video has %d frames' %len(rgb_input[0]))

The first video has 100 frames


In [26]:
rgb_input[0][0]

array([100.,  54.,  84., ...,  46.,  10., 132.])

In [27]:
rgb_input[0][0].shape

(1024,)

In [28]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import glob
import cv2

In [29]:
# x_data = np.array( [np.array(cv2.imread(rgb_input[0][0])) for i in range(len(rgb_input[0][0]))] )
# print (x_data.shape)

## Now were ready to create our model
****
** Since we're given sequential data, most likely I'll use an LSTM or GRU**

In [30]:
import keras
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.merge import dot, concatenate
from keras.layers import LSTM, Input, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU, PReLU

Using TensorFlow backend.


In [31]:
# audio_frame
X1 = Input(shape=(128,),name='audio_frame')
dense_1 = Dense(2048, activation='relu',name='dense_1')(X1)
batchNormalization_1 = BatchNormalization(name='batchNormalization_1')(dense_1)
leakyrelu_1 = LeakyReLU(alpha=.001, name='leakyrelu_1')(batchNormalization_1)
dropout_1 = Dropout(0.2, name='dropout_1')(leakyrelu_1)
dense_2 = Dense(2048, activation='relu',name='dense_2')(dropout_1)
batchNormalization_2 = BatchNormalization(name='batchNormalization_2')(dense_2)
leakyrelu_2 = LeakyReLU(alpha=.001, name='leakyrelu_2')(batchNormalization_2)
dropout_2 = Dropout(0.2, name='dropout_2')(leakyrelu_2)
dense_3 = Dense(2048, activation='relu',name='dense_3')(dropout_2)
# First Merge
merge_1 = keras.layers.Add(name='merge_1')([dense_3, dropout_1])
leakyrelu_3 = LeakyReLU(alpha=.001, name='leakyrelu_3')(merge_1)

# Complete Model Diagram
audio_frame_model = Model(inputs=[X1],outputs=[leakyrelu_3])
plot_model(audio_frame_model,to_file='audio_frame_model.png',show_shapes=True)

In [32]:
# rgb_frame
X2 = Input(shape=(1024,),name='rgb_frame')
dense_4 = Dense(2048, activation='relu',name='dense_4')(X2)
batchNormalization_3 = BatchNormalization(name='batchNormalization_3')(dense_4)
leakyrelu_4 = LeakyReLU(alpha=.001, name='leakyrelu_4')(batchNormalization_3)
dropout_3 = Dropout(0.2, name='dropout_3')(leakyrelu_4)
dense_5 = Dense(2048, activation='relu',name='dense_5')(dropout_3)
batchNormalization_4 = BatchNormalization(name='batchNormalization_4')(dense_5)
leakyrelu_5 = LeakyReLU(alpha=.001, name='leakyrelu_5')(batchNormalization_4)
dropout_4 = Dropout(0.2, name='dropout_4')(leakyrelu_5)
dense_6 = Dense(2048, activation='relu',name='dense_6')(dropout_4)
# Merging time
merge_2 = keras.layers.Add(name='merge_2')([dense_6, dropout_3])
leakyrelu_6 = LeakyReLU(alpha=.001, name='leakyrelu_6')(merge_2)

# Complete Model Diagram
rgb_frame_model = Model(inputs=[X2],outputs=[leakyrelu_6])
plot_model(rgb_frame_model,to_file='rgb_frame_model.png',show_shapes=True)

In [33]:
# Complete Model Diagram
# First merge between audio and rgb features
merge_audio_rgb = concatenate([leakyrelu_3, leakyrelu_6], name='merge_audio_rgb')
audio_rgb_model = Model(inputs=[X1,X2],outputs=[merge_audio_rgb])
plot_model(audio_rgb_model,to_file='audio_rgb_model.png',show_shapes=True)

In [34]:
dense_7 = Dense(2048, activation='relu',name='dense_7')(merge_audio_rgb)
batchNormalization_5 = BatchNormalization(name='batchNormalization_5')(dense_7)
leakyrelu_7 = LeakyReLU(alpha=.001, name='leakyrelu_7')(batchNormalization_5)
dropout_5 = Dropout(0.2, name='dropout_5')(leakyrelu_7)
dense_8 = Dense(4096, activation='relu',name='dense_8')(dropout_5)

# _model = Model(inputs=[merge_audio_rgb],outputs=[dense_8])
# plot_model(_model,to_file='_model.png',show_shapes=True)
# Merging time
merge_3 = keras.layers.Add(name='merge_3')([dense_8, merge_audio_rgb])

In [35]:
leakyrelu_8 = LeakyReLU(alpha=.001, name='leakyrelu_8')(merge_3)
dense_9 = Dense(2048, activation='relu',name='dense_9')(dropout_5)
batchNormalization_6 = BatchNormalization(name='batchNormalization_6')(dense_9)
leakyrelu_9 = LeakyReLU(alpha=.001, name='leakyrelu_9')(batchNormalization_6)
dropout_6 = Dropout(0.2, name='dropout_6')(leakyrelu_9)
dense_10 = Dense(4716, activation='relu',name='dense_10')(dropout_6)

In [36]:
# Complete Model Diagram
finalOutput = Model(inputs=[X1,X2],outputs=[dense_10])
plot_model(finalOutput,to_file='finalOutput.png',show_shapes=True)

In [37]:
finalOutput.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
print(finalOutput.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
audio_frame (InputLayer)        (None, 128)          0                                            
__________________________________________________________________________________________________
rgb_frame (InputLayer)          (None, 1024)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 2048)         264192      audio_frame[0][0]                
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 2048)         2099200     rgb_frame[0][0]                  
__________________________________________________________________________________________________
batchNorma

In [39]:
validation_data=({'audio_frame': X_test_audio,
                  'rgb_frame':X_test_rgb}, 
                 {'dense_10': y_test})

training_data = ({'audio_frame': X_train_audio,
                  'rgb_frame':X_train_rgb},
                 {'dense_10': y_train})

In [40]:
finalOutput.fit(
    training_data[0],
    training_data[1],
    epochs=60,
    batch_size=64,
    validation_data=validation_data)

ValueError: Error when checking input: expected audio_frame to have 2 dimensions, but got array with shape (4, 100, 128)

In [ ]:
# finalOutput.fit([X_train_audio, X_train_rgb], y_train, validation_data=([X_test_audio, X_test_rgb], y_test), epochs=60, batch_size=64)

In [ ]:
y_test.shape